## Extraction of patient diagnosis information from discharge summary using AWS Comprehend Medical

### Objective:

  -  Extract the ICU patients' discharge summaries from the MIMIC-III database
  -  The data is unstructured text data stored with the name of NOTEEVENTS
  -  The text data will be analyzed by AWS Comprehend Medical and we will extract the entity information
  -  We are interested to extract multiple diagnosis information and choose the top 10 diagnoses from each discharge summary.
  -  The information will be stored in a list format and will be clustered using the NLP Topic Modelling technique in the next notebook.
  -  The other tabular dataset is extracted from the ICU patients' lab-items records.
  -  Here we are dealing with heart failure patients who were admitted to ICU.
  -  

In [43]:
import pandas as pd

data_path = ''
# notes = pd.read_csv(note_path+'NOTEEVENTS.csv', skiprows= lambda x: x in [1289580])
lab_items = pd.read_csv(data_path+'lab_items_v01.csv')
print(lab_items.columns)

Index(['subject_id', 'hadm_id', 'admission_type', 'age', 'insurance',
       'ethnicity', 'religion', 'marital_status', 'gender', 'value',
       'valuenum', 'label', 'admittime', 'dischtime'],
      dtype='object')


In [44]:
import pandas as pd

note_path = ''
# note_path = '/media/tan/ssd-part-I/data/mimic3/1.4/not-related-to-CAD/'
# notes = pd.read_csv(note_path+'NOTEEVENTS.csv', skiprows= lambda x: x in [1289580])
notes_df = pd.read_csv(note_path+'noteevents_hf_v01.csv',low_memory=False)

In [45]:
def set_target(in_df):

    last_id = 0
    for d in in_df.index:
        if last_id == d[0]:
            # print(d)
            # (lab_items['dischtime'] - lab_items['admittime']).dt.days
            # disch_dt = np.datetime64(grouped_df.loc[(21,109451),['dischtime']].values[0])
            # admit_dt = np.datetime64(grouped_df.loc[(21,109451),['admittime']].values[0])
            dur = pd.Timedelta(in_df.loc[last_d,['dischtime']].values[0] - in_df.loc[d,['admittime']].values[0])
            if dur.days < 30:
                in_df.loc[last_d,['target']] = 1
            else:
                in_df.loc[last_d,['target']] = 0

        last_d = d
        last_id = d[0]
        
    
    return in_df

In [46]:
lab_items['dischtime'] = pd.to_datetime(lab_items['dischtime'],infer_datetime_format=True)
lab_items['admittime'] = pd.to_datetime(lab_items['admittime'],infer_datetime_format=True)

group_idx = ['subject_id', 'hadm_id', 'admission_type', 'age', 'insurance',
       'ethnicity', 'religion', 'marital_status', 'gender','label','admittime','dischtime']

grouped_df = lab_items.groupby(group_idx,as_index=False).apply(lambda x: round(x[['valuenum']].mean()))

grouped_df = grouped_df.pivot(index=['subject_id', 'hadm_id', 'admission_type', 'age', 'insurance', 'ethnicity', 'religion', 
                        'marital_status', 'gender','admittime','dischtime'],
                 columns='label',
                 values='valuenum').reset_index()

grouped_df = grouped_df.set_index(['subject_id','hadm_id'], drop = True)

grouped_df['target'] = 2

grouped_df = set_target(grouped_df.copy())

grouped_df['duration'] = (grouped_df['dischtime'] - grouped_df['admittime']).apply(lambda x: x.days)
grouped_df = grouped_df.drop(['admittime','dischtime'],axis=1)

grouped_df.fillna(0,inplace=True)

grouped_df = grouped_df.loc[~(grouped_df['target']==2)]

grouped_df

,label,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration
subject_id,hadm_id,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1


In [6]:
notes_df = notes_df.rename(columns={'SUBJECT_ID':'pat_id','HADM_ID':'adm_id'})
notes_df = notes_df.set_index(['pat_id','adm_id'])
notes_df = notes_df.rename_axis(['subject_id','hadm_id'])
notes_df

ROW_ID   CHARTDATE  CHARTTIME  STORETIME  \
subject_id hadm_id                                             
21         109451     7237  2134-09-24        NaN        NaN   
           111970     7238  2135-02-08        NaN        NaN   
68         108329     9139  2174-01-18        NaN        NaN   
           170467     9138  2174-01-03        NaN        NaN   
105        128744     1072  2189-02-25        NaN        NaN   
...                    ...         ...        ...        ...   
99556      196292      585  2167-07-31        NaN        NaN   
99650      158023     7324  2155-01-09        NaN        NaN   
           190854     7302  2152-06-12        NaN        NaN   
99897      162913    40065  2181-08-07        NaN        NaN   
           181057    40066  2182-07-08        NaN        NaN   

                             CATEGORY DESCRIPTION  CGID  ISERROR  \
subject_id hadm_id                                                 
21         109451   Discharge summary      Report   NaN      NaN   
           111970   Discharge summary      Report   NaN      NaN   
68         108329   Discharge summary      Report   NaN      NaN   
           170467   Discharge summary      Report   NaN      NaN   
105        128744   Discharge summary      Report   NaN      NaN   
...                               ...         ...   ...      ...   
99556      196292   Discharge summary      Report   NaN      NaN   
99650      158023   Discharge summary      Report   NaN      NaN   
           190854   Discharge summary      Report   NaN      NaN   
99897      162913   Discharge summary      Report   NaN      NaN   
           181057   Discharge summary      Report   NaN      NaN   

                                                                 TEXT  Column1  
subject_id hadm_id                                                              
21         109451   Admission Date:  [**2134-9-11**]              ...      NaN  
           111970   Admission Date:  [**2135-1-30**]              ...      NaN  
68         108329   Admission Date:  [**2174-1-4**]              D...      NaN  
           170467   Admission Date:  [**2173-12-15**]             ...      NaN  
105        128744   Admission Date:  [**2189-2-21**]              ...      NaN  
...                                                               ...      ...  
99556      196292   Admission Date:  [**2167-7-30**]              ...      NaN  
99650      158023   Admission Date:  [**2155-1-2**]              D...      NaN  
           190854   Admission Date: [**2152-6-7**]        Discharg...      NaN  
99897      162913   Admission Date:  [**2181-8-6**]              D...      NaN  
           181057   Admission Date:  [**2182-7-3**]              D...      NaN  

[5385 rows x 10 columns]

In [49]:
grouped_df

,label,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration
subject_id,hadm_id,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1


In [52]:
notes_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,Column1
0,7237,21,109451,2134-09-24,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-9-11**] ...,NaN
1,7238,21,111970,2135-02-08,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2135-1-30**] ...,NaN
2,9139,68,108329,2174-01-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2174-1-4**] D...,NaN
3,9138,68,170467,2174-01-03,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2173-12-15**] ...,NaN
4,1072,105,128744,2189-02-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2189-2-21**] ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5380,585,99556,196292,2167-07-31,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2167-7-30**] ...,NaN
5381,7324,99650,158023,2155-01-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2155-1-2**] D...,NaN
5382,7302,99650,190854,2152-06-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2152-6-7**] Discharg...,NaN
5383,40065,99897,162913,2181-08-07,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2181-8-6**] D...,NaN


In [53]:
notes_df = notes_df.rename(columns={'SUBJECT_ID':'pat_id','HADM_ID':'adm_id'})
notes_df = notes_df.set_index(['pat_id','adm_id'])
notes_df = notes_df.rename_axis(['subject_id','hadm_id'])
notes_df

ROW_ID   CHARTDATE  CHARTTIME  STORETIME  \
subject_id hadm_id                                             
21         109451     7237  2134-09-24        NaN        NaN   
           111970     7238  2135-02-08        NaN        NaN   
68         108329     9139  2174-01-18        NaN        NaN   
           170467     9138  2174-01-03        NaN        NaN   
105        128744     1072  2189-02-25        NaN        NaN   
...                    ...         ...        ...        ...   
99556      196292      585  2167-07-31        NaN        NaN   
99650      158023     7324  2155-01-09        NaN        NaN   
           190854     7302  2152-06-12        NaN        NaN   
99897      162913    40065  2181-08-07        NaN        NaN   
           181057    40066  2182-07-08        NaN        NaN   

                             CATEGORY DESCRIPTION  CGID  ISERROR  \
subject_id hadm_id                                                 
21         109451   Discharge summary      Report   NaN      NaN   
           111970   Discharge summary      Report   NaN      NaN   
68         108329   Discharge summary      Report   NaN      NaN   
           170467   Discharge summary      Report   NaN      NaN   
105        128744   Discharge summary      Report   NaN      NaN   
...                               ...         ...   ...      ...   
99556      196292   Discharge summary      Report   NaN      NaN   
99650      158023   Discharge summary      Report   NaN      NaN   
           190854   Discharge summary      Report   NaN      NaN   
99897      162913   Discharge summary      Report   NaN      NaN   
           181057   Discharge summary      Report   NaN      NaN   

                                                                 TEXT  Column1  
subject_id hadm_id                                                              
21         109451   Admission Date:  [**2134-9-11**]              ...      NaN  
           111970   Admission Date:  [**2135-1-30**]              ...      NaN  
68         108329   Admission Date:  [**2174-1-4**]              D...      NaN  
           170467   Admission Date:  [**2173-12-15**]             ...      NaN  
105        128744   Admission Date:  [**2189-2-21**]              ...      NaN  
...                                                               ...      ...  
99556      196292   Admission Date:  [**2167-7-30**]              ...      NaN  
99650      158023   Admission Date:  [**2155-1-2**]              D...      NaN  
           190854   Admission Date: [**2152-6-7**]        Discharg...      NaN  
99897      162913   Admission Date:  [**2181-8-6**]              D...      NaN  
           181057   Admission Date:  [**2182-7-3**]              D...      NaN  

[5385 rows x 10 columns]

In [54]:
grouped_df = grouped_df.merge(notes_df[['TEXT']],how='left',left_on=['subject_id','hadm_id'],right_on=['subject_id','hadm_id'])
grouped_df

,,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT
subject_id,hadm_id,,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,Admission Date: [**2134-9-11**] ...
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,NaN
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,Admission Date: [**2174-1-4**] D...
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,Admission Date: [**2189-2-21**] ...
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,Admission Date: [**2122-5-14**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4,NaN
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3,NaN
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1,Admission Date: [**2181-8-6**] D...


In [55]:
N = 10
dx_cols = ['dx_'+str(x) for x in range(1,N+1)]
dx_cols


['dx_1',
 'dx_2',
 'dx_3',
 'dx_4',
 'dx_5',
 'dx_6',
 'dx_7',
 'dx_8',
 'dx_9',
 'dx_10']

###  Below function will be used to connect AWS Comprehend Medical and extract the information from unstructured clinical text.



#### We are calling the Detect Entity function for Comprehend Medical 
  -  Extracting the entity text with type 'DX-NAME' which is diagnosis name
  -  Selecting top 10 diagnosis name mentioned in the discharge summary for each patient's encounter 
  

### Do not call this function as it has exhausted my AWS credits and moreover overshoot the billing ~4K dollar as it was running for ~6K discharge summaries and ran for several hours.

In [47]:

def comprih_extract(input_txt):
    import boto3
    import pandas as pd

    print('doc size', len(input_txt))
    
    input_txt = input_txt[0:20000]
    client = boto3.client(service_name='comprehendmedical',region_name='us-east-1')
    result = client.detect_entities_v2(Text=input_txt)
    tmp_df = pd.DataFrame(result['Entities'])

    return_val = list((tmp_df[tmp_df.Type=='DX_NAME']['Text'].value_counts()[0:10]).reset_index()['index'].values)
    
    if len(return_val) == 0:
        return_val += ['nan'] * (N-len(return_val)+1)
    else:
        return_val += ['nan'] * (N-len(return_val))

    print('length:',len(return_val))
    
    return return_val

In [56]:
N=9
l=['abc','kel']
l += ['nan']*(N-len(l))
# l1 = [2,4]
l

['abc', 'kel', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']

In [57]:
import re

grouped_df['TEXT'] = grouped_df['TEXT'].astype(str)
grouped_df['TEXT'] = grouped_df['TEXT'].str.lower()
grouped_df['TEXT'] = grouped_df['TEXT'].apply(lambda x: re.sub('[^a-z0-9-:]',' ',x)).values
grouped_df['TEXT'] = grouped_df['TEXT'].apply(lambda x: ' '.join(x.split())).values

In [58]:
grouped_df.columns

Index(['admission_type', 'age', 'insurance', 'ethnicity', 'religion',
       'marital_status', 'gender', 'Creatinine, Serum', 'NTproBNP', 'Sodium',
       'Urea Nitrogen', 'target', 'duration', 'TEXT'],
      dtype='object')

In [59]:
grouped_df.shape

(6377, 14)

In [60]:
grouped_df

,,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT
subject_id,hadm_id,,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,admission date: 2134-9-11 discharge date: 2134...
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,nan
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,admission date: 2174-1-4 discharge date: 2174-...
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,admission date: 2189-2-21 discharge date: 2189...
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,admission date: 2122-5-14 discharge date: 2122...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4,nan
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3,nan
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1,admission date: 2181-8-6 discharge date: 2181-...


###  Important**** Below function will be called to connect AWS Comprehend Medical and extract the information from unstructured clinical text.
 

### Do not call this function as it has exhausted my AWS credits and overshot the billing by ~4K dollars as it was running for ~6K discharge summaries and ran for several hours.

In [ ]:
# extract diagnosis code using AWS Comprehend Medical

#UNCOMMENT BEFORE USE 
#extract_df = pd.DataFrame(grouped_df['TEXT'].apply(comprih_extract))



####  After running comprih_extract() , Top 10 Dignosis names are extracted from discharge summary of the patients and stored in a CSV file

####  - The saved CSV file is used in the next step where diagnosis names are clustered in an unsupervised technique using NLP Topic Modelling.

####  - The clusters will be combined with tabular lab-items data and train the second model to check if accuracy increases.


In [ ]:
extract_df.to_csv("extract_df.csv")